# Једноставни класификатор слика

Овај нотебук вам показује како да класификујете слике користећи унапред обучену неуронску мрежу.

**Шта ћете научити:**
- Како учитати и користити унапред обучени модел
- Предобрада слика
- Прављење предвиђања на сликама
- Разумевање резултата поверења

**Пример употребе:** Идентификација објеката на сликама (као што су "мачка", "пас", "аутомобил" итд.)

---


## Корак 1: Увоз потребних библиотека

Хајде да увеземо алате који су нам потребни. Не брините ако још увек не разумете све ово!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Корак 2: Учитајте претходно обучени модел

Користићемо **MobileNetV2**, неуронску мрежу која је већ обучена на милионима слика.

Ово се назива **Трансферно учење** - коришћење модела који је неко други већ обучио!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Корак 3: Помоћне функције

Хајде да направимо функције за учитавање и припрему слика за наш модел.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Корак 4: Тестирање на примерима слика

Хајде да пробамо да класификујемо неке слике са интернета!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Класификујте сваку слику


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Корак 5: Испробајте своје слике!

Замените URL испод било којим URL-ом слике коју желите да класификујете.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Шта се управо догодило?

1. **Учитали смо унапред обучен модел** - MobileNetV2 је обучен на милионима слика  
2. **Претходно обрадили слике** - Променили величину и форматирали их за модел  
3. **Модел је направио предвиђања** - Добио је вероватноће за 1000 класа објеката  
4. **Декодирали смо резултате** - Претворили бројеве у људима разумљиве ознаке  

### Разумевање оцена поверења

- **90-100%**: Веома сигурно (готово сигурно тачно)  
- **70-90%**: Сигурно (вероватно тачно)  
- **50-70%**: Делимично сигурно (можда тачно)  
- **Испод 50%**: Није баш сигурно (несигурно)  

### Зашто предвиђања могу бити погрешна?

- **Необичан угао или осветљење** - Модел је обучен на типичним фотографијама  
- **Више објеката** - Модел очекује један главни објекат  
- **Ретки објекти** - Модел познаје само 1000 категорија  
- **Слика лошег квалитета** - Замагљене или пикселизоване слике су теже за обраду  

---


## 🚀 Следећи кораци

1. **Пробајте различите слике:**
   - Пронађите слике на [Unsplash](https://unsplash.com)
   - Десни клик → „Копирај адресу слике“ да добијете URL

2. **Експериментишите:**
   - Шта се дешава са апстрактном уметношћу?
   - Може ли препознати објекте из различитих углова?
   - Како се носи са више објеката?

3. **Сазнајте више:**
   - Истражите [лекције о рачунарском виду](../lessons/4-ComputerVision/README.md)
   - Научите како да обучите сопствени класификатор слика
   - Разумите како функционишу CNNs (Конволуционе неуронске мреже)

---

## 🎉 Честитамо!

Управо сте направили класификатор слика користећи најсавременију неуронску мрежу!

Иста техника покреће:
- Google Photos (организовање ваших фотографија)
- Аутомобиле који се сами возе (препознавање објеката)
- Медицинску дијагностику (анализу рендгенских снимака)
- Контролу квалитета (откривање недостатака)

Наставите да истражујете и учите! 🚀



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу произаћи из коришћења овог превода.
